https://automating-gis-processes.github.io/CSC18/lessons/L5/interactive-map-bokeh.html

In [1]:
from bokeh.palettes import YlOrRd as palette  #Spectral6 as palette
from bokeh.plotting import figure, save
from bokeh.models import ColumnDataSource, HoverTool, LogColorMapper
from bokeh.palettes import RdYlGn10 as palette 

In [2]:
import geopandas as gpd

In [3]:
import pysal as ps

C:\Users\paula.chiu\AppData\Local\Continuum\Anaconda2\envs\py3_geo\lib\site-packages\pysal\__init__.py:65: VisibleDeprecationWarning: PySAL's API will be changed on 2018-12-31. The last release made with this API is version 1.14.4. A preview of the next API version is provided in the `pysal` 2.0 prelease candidate. The API changes and a guide on how to change imports is provided at https://migrating.pysal.org
  ), VisibleDeprecationWarning)


In [4]:
# Filepaths
grid_fp = r"C:/repository/visualization/bokeh/helsinki/data/TravelTimes_to_5975375_RailwayStation.shp"
point_fp = r"C:/repository/visualization/bokeh/helsinki/data/addresses.shp"
metro_fp = r"C:/repository/visualization/bokeh/helsinki/data/metro.shp"

In [5]:
# Read files
grid = gpd.read_file(grid_fp)
points = gpd.read_file(point_fp)
metro = gpd.read_file(metro_fp)

In [6]:
# Get the CRS of our grid
CRS = grid.crs

In [7]:
print(CRS)

{'init': 'epsg:3067'}


In [8]:
# Convert the geometries of metro line and points into that one
points['geometry'] = points['geometry'].to_crs(crs=CRS)
metro['geometry'] = metro['geometry'].to_crs(crs=CRS)

In [9]:
points['geometry'].head(1)

0    POINT (385149.4478367225 6671962.669661295)
Name: geometry, dtype: object

In [10]:
metro['geometry'].head(1)

0    LINESTRING (395534.7026002127 6679490.08463068...
Name: geometry, dtype: object

In [11]:
grid['geometry'].head(1)

0    POLYGON ((382000.0001358641 6697750.000038058,...
Name: geometry, dtype: object

In [12]:
def getPointCoords(row, geom, coord_type):
    """Calculates coordinates ('x' or 'y') of a Point geometry"""
    if coord_type == 'x':
        return row[geom].x
    elif coord_type == 'y':
        return row[geom].y

In [13]:
def getLineCoords(row, geom, coord_type):
    """Returns a list of coordinates ('x' or 'y') of a LineString geometry"""
    if coord_type == 'x':
        return list( row[geom].coords.xy[0] )
    elif coord_type == 'y':
        return list( row[geom].coords.xy[1] )

In [14]:
def getPolyCoords(row, geom, coord_type):
    """Returns the coordinates ('x' or 'y') of edges of a Polygon exterior"""

    # Parse the exterior of the coordinate
    exterior = row[geom].exterior

    if coord_type == 'x':
        # Get the x coordinates of the exterior
        return list( exterior.coords.xy[0] )
    elif coord_type == 'y':
        # Get the y coordinates of the exterior
        return list( exterior.coords.xy[1] )

In [15]:
# Get the Polygon x and y coordinates
grid['x'] = grid.apply(getPolyCoords, geom='geometry', coord_type='x', axis=1)
grid['y'] = grid.apply(getPolyCoords, geom='geometry', coord_type='y', axis=1)

In [16]:
grid.head()

,car_m_d,car_m_t,car_r_d,car_r_t,from_id,pt_m_d,pt_m_t,pt_m_tt,pt_r_d,pt_r_t,pt_r_tt,to_id,walk_d,walk_t,geometry,x,y
0,32297,43,32260,48,5785640,32616,116,147,32616,108,139,5975375,32164,459,"POLYGON ((382000.0001358641 6697750.000038058,...","[382000.00013586413, 381750.0001359122, 381750...","[6697750.000038058, 6697750.000038066, 6698000..."
1,32508,43,32471,49,5785641,32822,119,145,32822,111,133,5975375,29547,422,"POLYGON ((382250.0001358146 6697750.000038053,...","[382250.0001358146, 382000.00013586413, 382000...","[6697750.000038053, 6697750.000038058, 6698000..."
2,30133,50,31872,56,5785642,32940,121,146,32940,113,133,5975375,29626,423,"POLYGON ((382500.0001357661 6697750.000038046,...","[382500.0001357661, 382250.0001358146, 382250....","[6697750.000038046, 6697750.000038053, 6698000..."
3,32690,54,34429,60,5785643,33233,125,150,33233,117,144,5975375,29919,427,"POLYGON ((382750.0001357181 6697750.000038039,...","[382750.0001357181, 382500.0001357661, 382500....","[6697750.000038039, 6697750.000038046, 6698000..."
4,31872,42,31834,48,5787544,32127,109,126,32127,101,121,5975375,31674,452,"POLYGON ((381250.0001360176 6697500.000038121,...","[381250.0001360176, 381000.00013606605, 381000...","[6697500.000038121, 6697500.0000381265, 669775..."


In [17]:
# Calculate x and y coordinates of the line
metro['x'] = metro.apply(getLineCoords, geom='geometry', coord_type='x', axis=1)
metro['y'] = metro.apply(getLineCoords, geom='geometry', coord_type='y', axis=1)

In [18]:
metro.head()

,NUMERO,SUUNTA,geometry,x,y
0,1300M,1,LINESTRING (395534.7026002127 6679490.08463068...,"[395534.70260021265, 394047.81013838784, 39396...","[6679490.084630681, 6679228.490560529, 6679185..."
1,1300M,2,LINESTRING (384398.5021810767 6671336.40736277...,"[384398.5021810767, 384626.1915761431, 384746....","[6671336.407362772, 6671348.068805486, 6671365..."
2,1300M1,1,LINESTRING (395534.7026002127 6679490.08463068...,"[395534.70260021265, 394047.81013838784, 39396...","[6679490.084630681, 6679228.490560529, 6679185..."
3,1300M1,2,LINESTRING (393658.5074443788 6676323.19948624...,"[393658.5074443788, 393811.1106914027, 393903....","[6676323.199486244, 6676400.316511797, 6676462..."
4,1300M2,1,LINESTRING (393658.5074443788 6676323.19948624...,"[393658.5074443788, 393352.974982422, 393185.8...","[6676323.199486244, 6676250.034046168, 6676206..."


In [19]:
# Calculate x and y coordinates of the points
points['x'] = points.apply(getPointCoords, geom='geometry', coord_type='x', axis=1)
points['y'] = points.apply(getPointCoords, geom='geometry', coord_type='y', axis=1)

In [20]:
# Show only head of x and y columns
grid[['x', 'y']].head()

,x,y
0,"[382000.00013586413, 381750.0001359122, 381750...","[6697750.000038058, 6697750.000038066, 6698000..."
1,"[382250.0001358146, 382000.00013586413, 382000...","[6697750.000038053, 6697750.000038058, 6698000..."
2,"[382500.0001357661, 382250.0001358146, 382250....","[6697750.000038046, 6697750.000038053, 6698000..."
3,"[382750.0001357181, 382500.0001357661, 382500....","[6697750.000038039, 6697750.000038046, 6698000..."
4,"[381250.0001360176, 381000.00013606605, 381000...","[6697500.000038121, 6697500.0000381265, 669775..."


In [21]:
# Replace No Data values (-1) with large number (999)
grid = grid.replace(-1, 999)

In [22]:
# Classify our travel times into 5 minute classes until 200 minutes
# Create a list of values where minumum value is 5, maximum value is 200 and step is 5.
breaks = [x for x in range(5, 200, 5)]

In [23]:
# Initialize the classifier and apply it
classifier = ps.User_Defined.make(bins=breaks)
# pt_classif = data[['pt_r_tt']].apply(classifier) # copied from the website
pt_classif = grid[['pt_r_tt']].apply(classifier)

In [24]:
pt_classif.head(1)

,pt_r_tt
0,27


In [25]:
# Rename the classified column
pt_classif.columns = ['pt_r_tt_ud']

In [26]:
pt_classif.head(1)

,pt_r_tt_ud
0,27


In [27]:
# Join it back to the grid layer
grid = grid.join(pt_classif)

In [28]:
grid.sample(5)

,car_m_d,car_m_t,car_r_d,car_r_t,from_id,pt_m_d,pt_m_t,pt_m_tt,pt_r_d,pt_r_t,pt_r_tt,to_id,walk_d,walk_t,geometry,x,y,pt_r_tt_ud
6247,13676,22,13676,25,5900207,12404,31,35,12404,31,35,5975375,12653,181,"POLYGON ((381750.0001363248 6682500.000040692,...","[381750.00013632484, 381500.0001363725, 381500...","[6682500.000040692, 6682500.000040698, 6682750...",6
11190,17088,39,17088,43,5960060,15466,58,65,18438,62,66,5975375,15170,217,"POLYGON ((373000.0001382663 6674250.00004236, ...","[373000.0001382663, 372750.0001383149, 372750....","[6674250.00004236, 6674250.000042367, 6674500....",13
13133,25609,43,25632,49,6004205,24343,76,91,25241,73,87,5975375,23193,331,"POLYGON ((374500.00013816 6667500.000043489, 3...","[374500.00013815996, 374250.0001382085, 374250...","[6667500.000043489, 6667500.0000435, 6667750.0...",17
5748,14133,23,14133,26,5894685,13166,35,39,13586,36,41,5975375,13357,191,"POLYGON ((381750.0001363041 6683250.000040561,...","[381750.0001363041, 381500.0001363518, 381500....","[6683250.000040561, 6683250.0000405675, 668350...",8
8703,23921,35,23928,40,5927712,19364,52,69,19202,53,74,5975375,19596,280,"POLYGON ((400000.0001328741 6678750.000040838,...","[400000.00013287406, 399750.0001329221, 399750...","[6678750.000040838, 6678750.000040847, 6679000...",14


In [29]:
grid[['pt_r_tt','pt_r_tt_ud']].head()

,pt_r_tt,pt_r_tt_ud
0,139,27
1,133,26
2,133,26
3,144,28
4,121,24


In [30]:
# Make a copy, drop the geometry column and create ColumnDataSource
m_df = metro.drop('geometry', axis=1).copy()
msource = ColumnDataSource(m_df)

In [31]:
# Make a copy, drop the geometry column and create ColumnDataSource
p_df = points.drop('geometry', axis=1).copy()
psource = ColumnDataSource(p_df)

In [32]:
# Make a copy, drop the geometry column and create ColumnDataSource
g_df = grid.drop('geometry', axis=1).copy()
gsource = ColumnDataSource(g_df)

### visualize

In [33]:
# Let's first do some coloring magic that converts the color palet into map numbers (it's okey not to understand)
from bokeh.palettes import RdYlBu11 as palette
from bokeh.models import LogColorMapper

# Create the color mapper
color_mapper = LogColorMapper(palette=palette)

In [34]:
# Specify the tools that we want to use
TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

In [35]:
# Initialize our figure
p = figure(title="Travel times to Helsinki city center by public transportation", tools=TOOLS,
           plot_width=650, plot_height=500, active_scroll = "wheel_zoom" )

# Do not add grid line
p.grid.grid_line_color = None

# # Plot grid
# p.patches('x', 'y', source=gsource,
#          fill_color={'field': 'pt_r_tt_ud', 'transform': color_mapper},
#          fill_alpha=1.0, line_color="black", line_width=0.05)

# Add polygon grid and a legend for it
grid = p.patches('x', 'y', source=gsource, name="grid",
         fill_color={'field': 'pt_r_tt_ud', 'transform': color_mapper},
         fill_alpha=1.0, line_color="black", line_width=0.03) # legend="label_pt"

# Add roads
r = p.multi_line('x', 'y', source=gsource, color="grey")

# # Add metro on top of the same figure
# p.multi_line('x', 'y', source=msource, color="red", line_width=2)

# Add metro
m = p.multi_line('x', 'y', source=msource, color="red", line_width=2)

# Add points on top (as black points)
p.circle('x', 'y', size=3, source=psource, color="black")

# Insert a circle on top of the Central Railway Station (coords in EurefFIN-TM35FIN)
station_x = 385752.214
station_y =  6672143.803
circle = p.circle(x=[station_x], y=[station_y], name="point", size=6, color="yellow")

# Add two separate hover tools for the data
phover = HoverTool(renderers=[circle])
phover.tooltips=[("Destination", "Railway Station")]

ghover = HoverTool(renderers=[grid])
ghover.tooltips=[("YKR-ID", "@from_id"),
                ("PT time", "@pt_r_tt"),
                ("Car time", "@car_r_t"),
                ]

p.add_tools(ghover)
p.add_tools(phover)

# Save the figure
outfp = r"C:/repository/visualization/bokeh/helsinki/travel_time_map.html"
save(p, outfp)

C:\Users\paula.chiu\AppData\Local\Continuum\Anaconda2\envs\py3_geo\lib\site-packages\bokeh\io\saving.py:123: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
C:\Users\paula.chiu\AppData\Local\Continuum\Anaconda2\envs\py3_geo\lib\site-packages\bokeh\io\saving.py:136: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")


'C:\\repository\\visualization\\bokeh\\helsinki\\travel_time_map.html'